In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import roc_auc_score

In [27]:
## read in data, came in 2 separate files 
# one contains features and the other contains the corresponding labels
features = pd.read_csv('training_set_features.csv')
labels = pd.read_csv('training_set_labels.csv')

## display the first 5 rows of both DataFrames
display(features.head())
labels.head()

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


,respondent_id,xyz_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0


In [28]:
df = pd.concat([features, labels.drop('respondent_id', axis=1)], axis=1)
df.head()

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,xyz_vaccine,seasonal_vaccine
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,0,0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,0,1
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,0,0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,0,1
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,0,0


In [29]:
df.count()

respondent_id                  26707
xyz_concern                    26615
xyz_knowledge                  26591
behavioral_antiviral_meds      26636
behavioral_avoidance           26499
behavioral_face_mask           26688
behavioral_wash_hands          26665
behavioral_large_gatherings    26620
behavioral_outside_home        26625
behavioral_touch_face          26579
doctor_recc_xyz                24547
doctor_recc_seasonal           24547
chronic_med_condition          25736
child_under_6_months           25887
health_worker                  25903
health_insurance               14433
opinion_xyz_vacc_effective     26316
opinion_xyz_risk               26319
opinion_xyz_sick_from_vacc     26312
opinion_seas_vacc_effective    26245
opinion_seas_risk              26193
opinion_seas_sick_from_vacc    26170
age_group                      26707
education                      25300
race                           26707
sex                            26707
income_poverty                 22284
m

In [30]:
df.drop(columns = ['respondent_id' ],axis =1 , inplace=True)

In [31]:
df.shape

(26707, 37)

In [32]:
df.isnull().sum()

xyz_concern                       92
xyz_knowledge                    116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_xyz                 2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_xyz_vacc_effective       391
opinion_xyz_risk                 388
opinion_xyz_sick_from_vacc       395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
marital_status                  1408
r

In [33]:
df.loc[df['employment_status'] == 'Unemployed', 'employment_industry'] = 'not employed'

## if a person is not in the labor force, change their 'employment_industry' to 'not_employed'
df.loc[df['employment_status'] == 'Not in Labor Force', 'employment_industry'] = 'not employed'



df.loc[df['employment_status'] == 'Unemployed', 'employment_occupation'] = 'not employed'

## if a person is not in the labor force, change their 'employment_industry' to 'not_employed'
df.loc[df['employment_status'] == 'Not in Labor Force', 'employment_occupation'] = 'not employed'



In [34]:
df.isnull().sum()

xyz_concern                       92
xyz_knowledge                    116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_xyz                 2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_xyz_vacc_effective       391
opinion_xyz_risk                 388
opinion_xyz_sick_from_vacc       395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
marital_status                  1408
r

In [35]:
df.shape

(26707, 37)

In [36]:
for col in df.columns:
    display(df[col].value_counts())

xyz_concern
2.0    10575
1.0     8153
3.0     4591
0.0     3296
Name: count, dtype: int64

xyz_knowledge
1.0    14598
2.0     9487
0.0     2506
Name: count, dtype: int64

behavioral_antiviral_meds
0.0    25335
1.0     1301
Name: count, dtype: int64

behavioral_avoidance
1.0    19228
0.0     7271
Name: count, dtype: int64

behavioral_face_mask
0.0    24847
1.0     1841
Name: count, dtype: int64

...

household_children
0.0    18672
1.0     3175
2.0     2864
3.0     1747
Name: count, dtype: int64

employment_industry
not employed    11684
fcxhlnwr         2468
wxleyezf         1804
ldnlellj         1231
pxcmvdjn         1037
atmlpfrs          926
arjwrbjb          871
xicduogh          851
mfikgejo          614
vjjrobsf          527
rucpziij          523
xqicxuve          511
saaquncn          338
cfqqtusy          325
nduyfdeo          286
mcubkhph          275
wlfvacwt          215
dotnnunm          201
haxffmxo          148
msuufmds          124
phxvnwax           89
qnlwzans           13
Name: count, dtype: int64

employment_occupation
not employed    11684
xtkaffoo         1778
mxkfnird         1509
emcorrxb         1270
cmhcxjea         1247
xgwztkwe         1082
hfxkjkmi          766
qxajmpny          548
xqwwgdyp          485
kldqjyjy          469
uqqtjvyb          452
tfqavkke          388
ukymxvdu          372
vlluhbov          354
oijqvulv          344
ccgxvspp          341
bxpfxfdn          331
haliazsg          296
rcertsgn          276
xzmlyyjv          248
dlvbwzss          227
hodpvpew          208
dcjcmpih          148
pvmttkik           98
Name: count, dtype: int64

xyz_vaccine
0    21033
1     5674
Name: count, dtype: int64

seasonal_vaccine
0    14272
1    12435
Name: count, dtype: int64

In [37]:
df.isnull().sum()

xyz_concern                       92
xyz_knowledge                    116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_xyz                 2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_xyz_vacc_effective       391
opinion_xyz_risk                 388
opinion_xyz_sick_from_vacc       395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
marital_status                  1408
r

In [40]:
cat_cols = df.select_dtypes(include=['object']).columns
num_cols = df.select_dtypes(exclude=['object']).columns


In [41]:
cat_cols

Index(['age_group', 'education', 'race', 'sex', 'income_poverty',
       'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region',
       'census_msa', 'employment_industry', 'employment_occupation'],
      dtype='object')

In [42]:
num_cols

Index(['xyz_concern', 'xyz_knowledge', 'behavioral_antiviral_meds',
       'behavioral_avoidance', 'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_xyz', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_xyz_vacc_effective', 'opinion_xyz_risk',
       'opinion_xyz_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'household_adults',
       'household_children', 'xyz_vaccine', 'seasonal_vaccine'],
      dtype='object')

In [63]:
df.isnull().sum()

xyz_concern                    0
xyz_knowledge                  0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_xyz                0
doctor_recc_seasonal           0
chronic_med_condition          0
child_under_6_months           0
health_worker                  0
health_insurance               0
opinion_xyz_vacc_effective     0
opinion_xyz_risk               0
opinion_xyz_sick_from_vacc     0
opinion_seas_vacc_effective    0
opinion_seas_risk              0
opinion_seas_sick_from_vacc    0
age_group                      0
education                      0
race                           0
sex                            0
income_poverty                 0
marital_status                 0
rent_or_own                    0
employment_status              0
hhs_geo_region                 0
census_msa

In [57]:
df.health_insurance.value_counts()

health_insurance
1.0    12697
0.0     1736
Name: count, dtype: int64

In [58]:
df['health_insurance'] = df['health_insurance'].fillna('missing')

In [60]:
columns_to_fill = [
    'income_poverty', 'education', 'marital_status', 'rent_or_own',
    'employment_status', 'employment_industry', 'employment_occupation',
    'doctor_recc_seasonal'
]

# Fill missing values with 'missing' in the specified columns
df[columns_to_fill] = df[columns_to_fill].fillna('missing')

In [62]:
numeric_cols = df.select_dtypes(include=[np.number]).columns

# Fill missing values in numeric columns with the median
df[numeric_cols] = df[numeric_cols].apply(lambda x: x.fillna(x.median()))


In [64]:
df.isna().sum()

xyz_concern                    0
xyz_knowledge                  0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_xyz                0
doctor_recc_seasonal           0
chronic_med_condition          0
child_under_6_months           0
health_worker                  0
health_insurance               0
opinion_xyz_vacc_effective     0
opinion_xyz_risk               0
opinion_xyz_sick_from_vacc     0
opinion_seas_vacc_effective    0
opinion_seas_risk              0
opinion_seas_sick_from_vacc    0
age_group                      0
education                      0
race                           0
sex                            0
income_poverty                 0
marital_status                 0
rent_or_own                    0
employment_status              0
hhs_geo_region                 0
census_msa

In [71]:
target = ['xyz_vaccine', 'seasonal_vaccine']
X = df.drop(columns=target, axis=1).copy()
y = df[target].copy()

## define random seed to use for train test split and later for classifiers for reproducibility
random_seed = 319

## split the data into training and test sets prior to preprocessing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=random_seed)

## check for class imbalance across all sets of y
print('**original**\n', y.value_counts(normalize=True), '\n------\n')
print('**y_train**\n', y_train.value_counts(normalize=True), '\n------\n')
print('**y_test**\n', y_test.value_counts(normalize=True), '\n------\n')

**original**
 xyz_vaccine  seasonal_vaccine
0            0                   0.497810
             1                   0.289737
1            1                   0.175871
             0                   0.036582
Name: proportion, dtype: float64 
------

**y_train**
 xyz_vaccine  seasonal_vaccine
0            0                   0.498902
             1                   0.288517
1            1                   0.175287
             0                   0.037294
Name: proportion, dtype: float64 
------

**y_test**
 xyz_vaccine  seasonal_vaccine
0            0                   0.494533
             1                   0.293395
1            1                   0.177625
             0                   0.034447
Name: proportion, dtype: float64 
------



In [77]:
# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(exclude=['object']).columns

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


In [89]:
print(X.dtypes)


xyz_concern                    float64
xyz_knowledge                  float64
behavioral_antiviral_meds      float64
behavioral_avoidance           float64
behavioral_face_mask           float64
behavioral_wash_hands          float64
behavioral_large_gatherings    float64
behavioral_outside_home        float64
behavioral_touch_face          float64
doctor_recc_xyz                float64
doctor_recc_seasonal            object
chronic_med_condition          float64
child_under_6_months           float64
health_worker                  float64
health_insurance                object
opinion_xyz_vacc_effective     float64
opinion_xyz_risk               float64
opinion_xyz_sick_from_vacc     float64
opinion_seas_vacc_effective    float64
opinion_seas_risk              float64
opinion_seas_sick_from_vacc    float64
age_group                       object
education                       object
race                            object
sex                             object
income_poverty           

In [92]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer



# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])


# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])



In [90]:
for i in categorical_cols:
    df[i] = df[i].astype(str)
    
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


In [94]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols)
    ])



In [87]:
def train_and_evaluate_multioutput(X_train, y_train, X_test, y_test):
    model = Pipeline(steps=[('preprocessor', preprocessor),
                            ('classifier', MultiOutputClassifier(LogisticRegression(max_iter=1000)))])

    param_grid = {
        'classifier__estimator__C': [0.1, 1, 10],
        'classifier__estimator__solver': ['liblinear', 'lbfgs']
    }

    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc_ovo', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict_proba(X_test)
    
    aucs = []
    for i, target in enumerate(target_variables):
        y_pred_prob = y_pred[i][:, 1]
        auc = roc_auc_score(y_test[target], y_pred_prob)
        aucs.append(auc)
        print(f'Best parameters for {target}: {grid_search.best_params_}')
        print(f'ROC AUC for {target}: {auc}')
    
    mean_auc = sum(aucs) / len(aucs)
    print(f'Mean ROC AUC: {mean_auc}')

    return best_model

In [99]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
model = train_and_evaluate_multioutput(X_train, y_train, X_test, y_test)

ValueError: 
All the 30 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\manav anand\anaconda3\Lib\site-packages\sklearn\utils\_encode.py", line 174, in _unique_python
    uniques = sorted(uniques_set)
              ^^^^^^^^^^^^^^^^^^^
TypeError: '<' not supported between instances of 'float' and 'str'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\manav anand\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\manav anand\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(

In [98]:
test_df = pd.read_csv('test.csv')
X_test_final = test_df


y_pred = model.predict_proba(X_test_final)

test_predictions = pd.DataFrame(test_df['respondent_id'], columns=['respondent_id'])
for i, target in enumerate(target):
    test_predictions[target] = y_pred[i][:, 1]

# Save predictions to CSV
test_predictions.to_csv('predictions.csv', index=False)

NameError: name 'model' is not defined